# Manual Frida Gadget Hooking Guide

This interactive notebook teaches you how to manually hook Android biometric authentication using Frida Gadget without automation scripts. Perfect for understanding the internals or customizing the process.

---

## Table of Contents

1. [Prerequisites](#prerequisites)
2. [Understanding the Architecture](#understanding-the-architecture)
3. [Step-by-Step Manual Process](#step-by-step-manual-process)
4. [Writing Custom Hooks](#writing-custom-hooks)
5. [Advanced Hooking Techniques](#advanced-hooking-techniques)
6. [Debugging Your Hooks](#debugging-your-hooks)
7. [Common Patterns](#common-patterns)
8. [Troubleshooting](#troubleshooting)

---

## Prerequisites

Before starting, you should have:

- ✅ Basic understanding of JavaScript
- ✅ Familiarity with Android architecture (Activities, APIs)
- ✅ Knowledge of smali bytecode (helpful but not required)
- ✅ APKTool installed and working
- ✅ Android device/emulator with root access
- ✅ ADB (Android Debug Bridge) set up
- ✅ Text editor for editing smali files

**Tools Needed:**
- [Frida Gadget](https://github.com/frida/frida/releases) (appropriate architecture)
- [APKTool](https://apktool.org/)
- Android SDK Build Tools (apksigner, zipalign)
- Text editor (VS Code, Sublime Text, etc.)

In [ ]:
# Check if required tools are installed
import subprocess
import sys

def check_tool(command, name):
    try:
        result = subprocess.run(command, capture_output=True, text=True, shell=True)
        print(f"✅ {name} is installed")
        return True
    except Exception as e:
        print(f"❌ {name} is NOT installed")
        return False

print("Checking prerequisites...\n")
check_tool("adb version", "ADB")
check_tool("apktool --version", "APKTool")
check_tool("zipalign", "zipalign")
check_tool("apksigner --version", "apksigner")

## Understanding the Architecture

### How Frida Gadget Works

```
┌─────────────────────────────────────────────┐
│          Android Application                │
├─────────────────────────────────────────────┤
│  MainActivity.smali                         │
│  ┌──────────────────────────────┐          │
│  │  System.loadLibrary("frida-gadget") │  │
│  └────────────┬─────────────────┘          │
│               ↓                             │
│  ┌─────────────────────────────────┐       │
│  │  libfrida-gadget.so              │       │
│  │  (Frida Runtime)                 │       │
│  └────────────┬─────────────────────┘       │
│               ↓                             │
│  ┌─────────────────────────────────┐       │
│  │  libfrida-gadget.config.so       │       │
│  │  (Configuration)                 │       │
│  └────────────┬─────────────────────┘       │
│               ↓                             │
│  ┌─────────────────────────────────┐       │
│  │  libfrida-gadget.script.so       │       │
│  │  (Your JavaScript Hooks)         │       │
│  └──────────────────────────────────┘       │
└─────────────────────────────────────────────┘
```

### The Three Files

1. **libfrida-gadget.so**: The Frida runtime engine
2. **libfrida-gadget.config.so**: JSON configuration (tells Gadget what to do)
3. **libfrida-gadget.script.so**: Your JavaScript hook code

## Step-by-Step Manual Process

### Step 1: Decompile the APK

In [ ]:
# Set your APK path here
APK_PATH = "app.apk"  # Change this to your APK path
OUTPUT_DIR = "app_decompiled"

# Decompile command
decompile_cmd = f"apktool d {APK_PATH} -o {OUTPUT_DIR}"
print(f"Run this command to decompile:\n{decompile_cmd}")

# Uncomment to execute
# !{decompile_cmd}

**What this does:**
- Extracts the APK contents
- Converts DEX bytecode to smali
- Creates an editable project structure

### Step 2: Locate MainActivity

In [ ]:
import os
import glob

# Find MainActivity.smali
def find_mainactivity(base_dir="app_decompiled"):
    pattern = os.path.join(base_dir, "**/MainActivity.smali")
    matches = glob.glob(pattern, recursive=True)
    
    if matches:
        print("Found MainActivity.smali at:")
        for match in matches:
            print(f"  - {match}")
        return matches[0]
    else:
        print("MainActivity.smali not found. Check AndroidManifest.xml for the main activity name.")
        return None

# Uncomment to search
# mainactivity_path = find_mainactivity()

**Common locations:**
- `smali/com/company/app/MainActivity.smali`
- `smali_classes2/com/company/app/MainActivity.smali`
- `smali_classes3/com/company/app/MainActivity.smali`

### Step 3: Inject Gadget Loader (Manual Method)

Open `MainActivity.smali` in your text editor and find the `onCreate` method:

```smali
.method protected onCreate(Landroid/os/Bundle;)V
    .locals 1
    .param p1, "savedInstanceState"    # Landroid/os/Bundle;

    .line 15
    invoke-super {p0, p1}, Landroidx/appcompat/app/AppCompatActivity;->onCreate(Landroid/os/Bundle;)V

    # YOUR CODE GOES HERE

    .line 16
    const v0, 0x7f0b001c
    invoke-virtual {p0, v0}, Lcom/example/app/MainActivity;->setContentView(I)V

    return-void
.end method
```

**Add the Gadget loader AFTER `invoke-super` but BEFORE any other code:**

```smali
.method protected onCreate(Landroid/os/Bundle;)V
    .locals 1
    .param p1, "savedInstanceState"    # Landroid/os/Bundle;

    .line 15
    invoke-super {p0, p1}, Landroidx/appcompat/app/AppCompatActivity;->onCreate(Landroid/os/Bundle;)V

    # ===== FRIDA GADGET LOADER START =====
    const-string v0, "frida-gadget"
    invoke-static {v0}, Ljava/lang/System;->loadLibrary(Ljava/lang/String;)V
    # ===== FRIDA GADGET LOADER END =====

    .line 16
    const v0, 0x7f0b001c
    invoke-virtual {p0, v0}, Lcom/example/app/MainActivity;->setContentView(I)V

    return-void
.end method
```

In [ ]:
# Helper function to inject Frida Gadget loader
def inject_gadget_loader(mainactivity_path):
    """
    Injects Frida Gadget loader into MainActivity.smali
    """
    GADGET_LOADER = '''    # ===== FRIDA GADGET LOADER START =====
    const-string v0, "frida-gadget"
    invoke-static {v0}, Ljava/lang/System;->loadLibrary(Ljava/lang/String;)V
    # ===== FRIDA GADGET LOADER END =====
'''
    
    with open(mainactivity_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Find onCreate and inject after invoke-super
    # This is a simplified version - production code should be more robust
    if 'FRIDA GADGET LOADER' in content:
        print("⚠️ Gadget loader already injected")
        return False
    
    # Look for invoke-super in onCreate
    lines = content.split('\n')
    modified = False
    new_lines = []
    
    for i, line in enumerate(lines):
        new_lines.append(line)
        if 'invoke-super' in line and 'onCreate' in ''.join(lines[max(0, i-10):i]):
            new_lines.append(GADGET_LOADER)
            modified = True
            break
    
    if modified:
        new_lines.extend(lines[len(new_lines):])
        with open(mainactivity_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(new_lines))
        print("✅ Gadget loader injected successfully")
        return True
    else:
        print("❌ Could not find injection point")
        return False

# Uncomment to use
# inject_gadget_loader("app_decompiled/smali/com/example/app/MainActivity.smali")

### Step 4: Modify AndroidManifest.xml

Find the `<application>` tag and add `android:extractNativeLibs="true"`:

In [ ]:
import xml.etree.ElementTree as ET

def patch_manifest(manifest_path="app_decompiled/AndroidManifest.xml"):
    """
    Adds android:extractNativeLibs="true" to AndroidManifest.xml
    """
    # Read the manifest
    with open(manifest_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    if 'extractNativeLibs' in content:
        print("⚠️ extractNativeLibs already set")
        return
    
    # Find <application> tag and add attribute
    content = content.replace(
        '<application',
        '<application\n        android:extractNativeLibs="true"'
    )
    
    with open(manifest_path, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print("✅ AndroidManifest.xml patched")

# Uncomment to use
# patch_manifest()

**Why?** Android 9+ doesn't extract `.so` files by default. This forces extraction so Gadget can load.

### Step 5: Prepare Frida Gadget Files

#### 5a. Download Frida Gadget

In [ ]:
# Download Frida Gadget (x86_64 for emulator)
FRIDA_VERSION = "16.5.9"
ARCHITECTURE = "x86_64"  # or "arm64" for physical devices

download_url = f"https://github.com/frida/frida/releases/download/{FRIDA_VERSION}/frida-gadget-{FRIDA_VERSION}-android-{ARCHITECTURE}.so.xz"

print(f"Download Frida Gadget from:\n{download_url}\n")
print("Commands to download and extract:")
print(f"wget {download_url}")
print(f"xz -d frida-gadget-{FRIDA_VERSION}-android-{ARCHITECTURE}.so.xz")
print(f"mv frida-gadget-{FRIDA_VERSION}-android-{ARCHITECTURE}.so libfrida-gadget.so")

#### 5b. Create Configuration File

In [ ]:
import json

# Create Frida Gadget configuration
config = {
    "interaction": {
        "type": "script",
        "path": "libfrida-gadget.script.so",
        "on_load": "init"
    }
}

config_json = json.dumps(config, indent=2)
print("libfrida-gadget.config.so content:")
print(config_json)

# Save to file
with open("libfrida-gadget.config.so", "w") as f:
    f.write(config_json)

print("\n✅ Configuration file created")

**Configuration Breakdown:**
- `"type": "script"` → Load a script file
- `"path": "libfrida-gadget.script.so"` → Script filename
- `"on_load": "init"` → Call `init()` function when loaded

### Step 6: Write Your Hook Script

In [ ]:
# JavaScript hook script
hook_script = '''
// ===== BioShield Biometric Hook =====
// Hooks androidx.biometric.BiometricPrompt

function init() {
    console.log("=== [BioShield] init() called ===");

    // Wait for Java to be ready
    Java.perform(function() {
        console.log("[BioShield] Java.perform() started");

        try {
            // Get BiometricPrompt class
            var BiometricPrompt = Java.use('androidx.biometric.BiometricPrompt');
            console.log("[OK] Found BiometricPrompt class");

            // Hook authenticate() method
            BiometricPrompt.authenticate.overload(
                'androidx.biometric.BiometricPrompt$PromptInfo'
            ).implementation = function(promptInfo) {
                console.log("[HOOK] BiometricPrompt.authenticate() called");
                console.log("  PromptInfo: " + promptInfo);

                // Get timestamp
                var timestamp = new Date().toISOString();
                console.log("  Timestamp: " + timestamp);

                // Call original method
                var result = this.authenticate(promptInfo);

                return result;
            };

            console.log("[OK] Hooked BiometricPrompt.authenticate()");

            // Hook AuthenticationCallback
            var Callback = Java.use('androidx.biometric.BiometricPrompt$AuthenticationCallback');

            // Hook onAuthenticationSucceeded
            Callback.onAuthenticationSucceeded.implementation = function(result) {
                console.log("[EVENT] ✓ Authentication SUCCEEDED");
                console.log("  Result: " + result);

                // Log to file
                logEvent({
                    event: 'SUCCESS',
                    timestamp: new Date().toISOString(),
                    result: String(result)
                });

                // Call original
                this.onAuthenticationSucceeded(result);
            };

            // Hook onAuthenticationFailed
            Callback.onAuthenticationFailed.implementation = function() {
                console.log("[EVENT] ✗ Authentication FAILED");

                logEvent({
                    event: 'FAILED',
                    timestamp: new Date().toISOString()
                });

                this.onAuthenticationFailed();
            };

            // Hook onAuthenticationError
            Callback.onAuthenticationError.implementation = function(errorCode, errString) {
                console.log("[EVENT] ⚠ Authentication ERROR");
                console.log("  Code: " + errorCode);
                console.log("  Message: " + errString);

                logEvent({
                    event: 'ERROR',
                    timestamp: new Date().toISOString(),
                    errorCode: errorCode,
                    errorMessage: String(errString)
                });

                this.onAuthenticationError(errorCode, errString);
            };

            console.log("=== [BioShield] Hooks Active ===");

        } catch (error) {
            console.error("[ERROR] Hook failed: " + error);
        }
    });
}

// Helper function to log events to file
function logEvent(data) {
    var File = Java.use('java.io.File');
    var FileWriter = Java.use('java.io.FileWriter');

    try {
        // Create directory
        var dir = File.$new('/storage/emulated/0/Download/BioShield');
        if (!dir.exists()) {
            dir.mkdirs();
        }

        // Create log file
        var logFile = File.$new(dir, 'logs.jsonl');
        var writer = FileWriter.$new(logFile, true); // append mode

        // Write JSON line
        writer.write(JSON.stringify(data) + '\\n');
        writer.close();

        console.log("[LOG] Written to: " + logFile.getAbsolutePath());
    } catch (e) {
        console.error("[ERROR] Log write failed: " + e);
    }
}

// Export init function
rpc.exports = {
    init: init
};
'''

# Save to file
with open("libfrida-gadget.script.so", "w") as f:
    f.write(hook_script)

print("✅ Hook script created: libfrida-gadget.script.so")
print(f"Script length: {len(hook_script)} characters")

**Key Concepts:**

1. **`Java.perform()`** → Ensures Java VM is ready
2. **`Java.use()`** → Gets a Java class reference
3. **`.implementation`** → Replaces method implementation
4. **`this.originalMethod()`** → Calls the original method

### Step 7: Place Files in Correct Location

In [ ]:
import shutil

def place_gadget_files(decompiled_dir="app_decompiled", architecture="x86_64"):
    """
    Places Frida Gadget files in the correct lib/ folder
    """
    lib_dir = os.path.join(decompiled_dir, "lib", architecture)
    os.makedirs(lib_dir, exist_ok=True)
    
    files = [
        "libfrida-gadget.so",
        "libfrida-gadget.config.so",
        "libfrida-gadget.script.so"
    ]
    
    for file in files:
        if os.path.exists(file):
            dest = os.path.join(lib_dir, file)
            shutil.copy(file, dest)
            print(f"✅ Copied {file} to {dest}")
        else:
            print(f"❌ {file} not found")
    
    print(f"\nAll files placed in: {lib_dir}")

# Uncomment to use
# place_gadget_files()

**⚠️ Critical:** All three files MUST end with `.so` extension!

### Step 8: Rebuild and Sign APK

In [ ]:
# Rebuild, align, and sign APK
DECOMPILED_DIR = "app_decompiled"
REPACKED_APK = "app_repacked.apk"
ALIGNED_APK = "app_aligned.apk"
FINAL_APK = "app_final.apk"

print("Commands to rebuild and sign:")
print()
print(f"# 1. Rebuild APK")
print(f"apktool b {DECOMPILED_DIR} -o {REPACKED_APK}")
print()
print(f"# 2. Align APK")
print(f"zipalign -f -v 4 {REPACKED_APK} {ALIGNED_APK}")
print()
print(f"# 3. Sign APK with debug keystore")
print(f"apksigner sign --ks ~/.android/debug.keystore \\")
print(f"    --ks-pass pass:android \\")
print(f"    --key-pass pass:android \\")
print(f"    --min-sdk-version 21 \\")
print(f"    --out {FINAL_APK} \\")
print(f"    {ALIGNED_APK}")
print()
print(f"# 4. Verify signature")
print(f"apksigner verify {FINAL_APK}")

### Step 9: Install and Test

In [ ]:
# Commands to install and monitor
PACKAGE_NAME = "com.example.app"  # Change this

print("Commands to install and test:")
print()
print(f"# 1. Uninstall old version")
print(f"adb uninstall {PACKAGE_NAME}")
print()
print(f"# 2. Install new version")
print(f"adb install {FINAL_APK}")
print()
print(f"# 3. Monitor logs")
print(f'adb logcat | grep -E "FRIDA|BioShield"')

**Expected Output:**

```
I FRIDA: Frida Gadget loaded
I BioShield: === [BioShield] init() called ===
I BioShield: [BioShield] Java.perform() started
I BioShield: [OK] Found BiometricPrompt class
I BioShield: [OK] Hooked BiometricPrompt.authenticate()
I BioShield: === [BioShield] Hooks Active ===
```

**When you trigger biometric authentication:**

```
I BioShield: [HOOK] BiometricPrompt.authenticate() called
I BioShield: [EVENT] ✓ Authentication SUCCEEDED
I BioShield: [LOG] Written to: /storage/emulated/0/Download/BioShield/logs.jsonl
```

## Writing Custom Hooks

### Basic Hook Template

In [ ]:
%%javascript
// Basic hook template
Java.perform(function() {
    var ClassName = Java.use('com.example.ClassName');

    ClassName.methodName.implementation = function(arg1, arg2) {
        console.log("methodName called with: " + arg1 + ", " + arg2);

        // Call original
        var result = this.methodName(arg1, arg2);

        console.log("methodName returned: " + result);
        return result;
    };
});

### Overloading Methods

If a method has multiple signatures, specify the overload:

In [ ]:
%%javascript
// Method with no arguments
ClassName.methodName.overload().implementation = function() {
    console.log("No-arg version called");
    return this.methodName();
};

// Method with String argument
ClassName.methodName.overload('java.lang.String').implementation = function(str) {
    console.log("String version called: " + str);
    return this.methodName(str);
};

// Method with multiple arguments
ClassName.methodName.overload('int', 'java.lang.String').implementation = function(num, str) {
    console.log("Multi-arg version called");
    return this.methodName(num, str);
};

## Advanced Hooking Techniques

### 1. Hooking Constructors

In [ ]:
%%javascript
var BiometricPrompt = Java.use('androidx.biometric.BiometricPrompt');

BiometricPrompt.$init.overload(
    'androidx.fragment.app.FragmentActivity',
    'androidx.biometric.BiometricPrompt$AuthenticationCallback'
).implementation = function(activity, callback) {
    console.log("[CONSTRUCTOR] BiometricPrompt created");
    console.log("  Activity: " + activity);
    console.log("  Callback: " + callback);

    // Call original constructor
    this.$init(activity, callback);
};

### 2. Tracing All Methods

In [ ]:
%%javascript
function traceClass(className) {
    var targetClass = Java.use(className);
    var methods = targetClass.class.getDeclaredMethods();

    methods.forEach(function(method) {
        var methodName = method.getName();
        console.log("[TRACE] Found method: " + methodName);

        // Hook each method
        try {
            targetClass[methodName].implementation = function() {
                console.log("[CALL] " + className + "." + methodName);
                return this[methodName].apply(this, arguments);
            };
        } catch (e) {
            // Some methods can't be hooked (overloaded, etc.)
        }
    });
}

// Use it
traceClass('androidx.biometric.BiometricPrompt');

## Debugging Your Hooks

### 1. Check if Gadget Loaded

In [ ]:
# Check if Frida Gadget loaded
print("Run this command to check if Gadget loaded:")
print('adb logcat | grep -i "frida"')
print()
print("Expected output:")
print("I FRIDA: Frida Gadget loaded successfully")

### 2. Check if Script Loaded

In [ ]:
print("Run this command to check if script loaded:")
print('adb logcat | grep -i "bioshield\\|init"')
print()
print("Expected output:")
print("I BioShield: === [BioShield] init() called ===")

### 3. Check Hook Execution

In [ ]:
print("Run this command to check hook execution:")
print('adb logcat | grep -E "HOOK|EVENT"')
print()
print("Expected output (when biometric is triggered):")
print("I BioShield: [HOOK] BiometricPrompt.authenticate() called")
print("I BioShield: [EVENT] ✓ Authentication SUCCEEDED")

## Common Patterns

### Pattern 1: Timing Attack Detection

In [ ]:
%%javascript
var startTime = 0;

BiometricPrompt.authenticate.implementation = function(promptInfo) {
    startTime = Date.now();
    console.log("[START] Auth started at: " + startTime);

    return this.authenticate(promptInfo);
};

Callback.onAuthenticationSucceeded.implementation = function(result) {
    var endTime = Date.now();
    var duration = endTime - startTime;

    console.log("[TIMING] Auth took: " + duration + "ms");

    if (duration < 100) {
        console.log("[ALERT] Suspiciously fast authentication!");
    }

    this.onAuthenticationSucceeded(result);
};

### Pattern 2: Replay Attack Detection

In [ ]:
%%javascript
var usedNonces = new Set();

BiometricPrompt.authenticate.implementation = function(promptInfo) {
    var nonce = Java.use('java.util.UUID').randomUUID().toString();

    if (usedNonces.has(nonce)) {
        console.log("[ALERT] Replay attack detected!");
    }

    usedNonces.add(nonce);
    console.log("[NONCE] Using: " + nonce);

    return this.authenticate(promptInfo);
};

## Troubleshooting

### Problem: App crashes on startup

**Symptoms:**
```
E AndroidRuntime: FATAL EXCEPTION: main
E AndroidRuntime: java.lang.UnsatisfiedLinkError: dlopen failed
```

**Solutions:**
1. Check `android:extractNativeLibs="true"` in manifest
2. Verify `.so` files are in correct architecture folder
3. Ensure all 3 files end with `.so` extension

---

### Problem: Gadget loads but script doesn't run

**Symptoms:**
```
I FRIDA: Frida Gadget loaded
(but no [BioShield] logs)
```

**Solutions:**
1. Check `libfrida-gadget.config.so` has correct JSON
2. Verify `"on_load": "init"` is present
3. Check script has `function init() {...}`
4. Look for JavaScript errors in logcat

---

### Problem: Hooks don't trigger

**Symptoms:**
```
I BioShield: === [BioShield] Hooks Active ===
(but no [HOOK] or [EVENT] logs when using biometric)
```

**Solutions:**
1. App might use different biometric API
2. Check class name is correct: `androidx.biometric.BiometricPrompt`
3. Device might not have fingerprint enrolled
4. Add logging to verify Java.perform() completed

## Summary

### Complete Workflow Checklist

- [ ] 1. Decompile APK with APKTool
- [ ] 2. Locate MainActivity.smali
- [ ] 3. Inject Frida Gadget loader in onCreate()
- [ ] 4. Add extractNativeLibs="true" to AndroidManifest.xml
- [ ] 5. Download and prepare Frida Gadget binary
- [ ] 6. Create libfrida-gadget.config.so
- [ ] 7. Write libfrida-gadget.script.so hook script
- [ ] 8. Place all 3 .so files in lib/<architecture>/
- [ ] 9. Rebuild APK with APKTool
- [ ] 10. Align with zipalign
- [ ] 11. Sign with apksigner
- [ ] 12. Install and test
- [ ] 13. Monitor logcat for hook execution

### Next Steps

Now that you understand manual hooking:

1. **Customize the script** for your specific needs
2. **Add encryption** to protect logged data
3. **Implement side-channel detection** (timing, replay attacks)
4. **Create reusable hook modules** for different APIs
5. **Learn smali editing** for deeper modifications


---

**Questions?** Refer to the [Frida documentation](https://frida.re/docs/).

**Version:** 1.0  
**Last Updated:** 2025-01-20  
**Author:** BioShield Team